<a target="_blank" href="https://colab.research.google.com/drive/1fFUsO8xnx7qOH6VCHdO-MzEN_6geDxZK?usp=sharing">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

In [1]:
%%capture
! pip install pip==21.3.1
! pip install ultralytics==8.0.53
! pip install clearml==1.9.3

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
from glob import glob
import numpy as np
import torch
import ultralytics
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import clearml
from clearml import Task

In [5]:
class cnf:
    PROJECT_NAME = "Pedestrian-Detection-YOLOv8"
    TASK_NAME = "yolov8n_imgsz640_epochs200"
    CKPT = "yolov8n.pt"
    IMGSZ = 640
    EPOCHS = 200

In [6]:
ultralytics.checks()

Ultralytics YOLOv8.0.53 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
Setup complete ✅ (12 CPUs, 83.5 GB RAM, 25.5/166.8 GB disk)


In [7]:
clearml.browser_login()

<IPython.core.display.Javascript object>


🤖 ClearML connected successfully - let's build something! 🚀


In [8]:
%%shell
cp /content/drive/MyDrive/Reza/Projects/Pedestrian-Detection/data/citypersons.zip .
unzip -q citypersons.zip

In [9]:
# create ClearML experiment and task
task = Task.init(project_name=cnf.PROJECT_NAME, task_name=cnf.TASK_NAME)

ClearML Task: created new task id=0bcc5833468c46d1a2f254c73f3f0ee1
ClearML results page: https://app.clear.ml/projects/c44ac5edda124f9fa0400f27bf7659c4/experiments/0bcc5833468c46d1a2f254c73f3f0ee1/output/log
2023-03-17 10:24:27,142 - clearml.Task - INFO - Storing jupyter notebook directly as code


In [10]:
# load a pretrained model
model = YOLO(cnf.CKPT)

  0%|          | 0.00/6.23M [00:00<?, ?B/s]

2023-03-17 10:24:30,680 - clearml.model - INFO - Selected model id: 96920760883b420098dc25cc39edaf86


In [11]:
%%time
# train the model
model.train(data='/content/citypersons/dataset.yaml', 
            imgsz=cnf.IMGSZ, 
            epochs=cnf.EPOCHS)

New https://pypi.org/project/ultralytics/8.0.54 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.53 🚀 Python-3.9.16 torch-1.13.1+cu116 CUDA:0 (NVIDIA A100-SXM4-40GB, 40514MiB)
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/citypersons/dataset.yaml, epochs=200, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, classes

  0%|          | 0.00/755k [00:00<?, ?B/s]

Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.Conv                  [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.Conv                  [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.C2f                   [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.Conv                  [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.C2f                   [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.Conv                  [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.C2f                   [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

2023-03-17 10:26:05,981 - clearml.frameworks - INFO - Found existing registered model id=3f6fb710d3634d36ad6e029b3808e613 [/content/runs/detect/train/weights/last.pt] reusing it.
2023-03-17 10:26:16,699 - clearml.frameworks - INFO - Found existing registered model id=e962754b91194f3c8c9a912133edcac3 [/content/runs/detect/train/weights/best.pt] reusing it.



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      2/200      3.29G      1.732      1.486      1.053         95        640: 100%|██████████| 160/160 [00:40<00:00,  3.97it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.08it/s]
                   all        451       4164      0.509      0.319      0.335      0.178

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      3/200      3.29G      1.772      1.492      1.061         90        640: 100%|██████████| 160/160 [00:40<00:00,  3.96it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:04<00:00,  3.09it/s]
                   all        451       4164      0.545      0.302      0.313      0.167

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size
      4/200      3.29G      1.746      1.488      1

2023-03-17 13:31:23,970 - clearml.Task - INFO - Completed model upload to https://files.clear.ml/Pedestrian-Detection-YOLOv8/yolov8n_imgsz640_epochs200.0bcc5833468c46d1a2f254c73f3f0ee1/models/best.pt
CPU times: user 1h 20min 50s, sys: 12min 9s, total: 1h 32min 59s
Wall time: 3h 6min 46s


In [14]:
# close clearml task
task.close()

In [12]:
# helper functions for ploting predictions
def box_label(image, box, label='', color=(128, 128, 128), txt_color=(255, 255, 255)):
  lw = max(round(sum(image.shape) / 2 * 0.003), 2)
  p1, p2 = (int(box[0]), int(box[1])), (int(box[2]), int(box[3]))
  cv2.rectangle(image, p1, p2, color, thickness=lw, lineType=cv2.LINE_AA)
  if label:
    tf = max(lw - 1, 1)  # font thickness
    w, h = cv2.getTextSize(label, 0, fontScale=lw / 3, thickness=tf)[0]  # text width, height
    outside = p1[1] - h >= 3
    p2 = p1[0] + w, p1[1] - h - 3 if outside else p1[1] + h + 3
    cv2.rectangle(image, p1, p2, color, -1, cv2.LINE_AA)  # filled
    cv2.putText(image,
                label, (p1[0], p1[1] - 2 if outside else p1[1] + h + 2),
                0,
                lw / 3,
                txt_color,
                thickness=tf,
                lineType=cv2.LINE_AA)
    
def plot_bboxes(image, boxes, labels=[], colors=[], score=True, conf=None):
  # Define Labels
  if labels == []:
    labels = {0: u'__background__', 1: u'rider', 2: u'pedestrian'}
  # Define colors
  if colors == []:
    
    colors = [(89, 161, 197), (190, 76, 98), (130, 172, 179), 
              (67, 161, 255),(19, 222, 24),(186, 55, 2),
              (167, 146, 11),(130, 172, 179),(115, 209, 128),
              (204, 79, 135),(136, 126, 185),(209, 213, 45),]              
  
  #plot each boxes
  for box in boxes:
    #add score in label if score=True
    if score :
      label = labels[int(box[-1])+1] + " " + str(round(100 * float(box[-2]),1)) + "%"
    else :
      label = labels[int(box[-1])+1]
    #filter every box under conf threshold if conf threshold setted
    if conf :
      if box[-2] > conf:
        color = colors[int(box[-1])]
        box_label(image, box, label, color)
    else:
      color = colors[int(box[-1])]
      box_label(image, box, label, color)

  #show image
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  try:
    import google.colab
    IN_COLAB = True
  except:
    IN_COLAB = False

  if IN_COLAB:
    cv2_imshow(image) #if used in Colab
  else :
    cv2.imshow(image) #if used in Python

In [13]:
# make prediction on a few validation images and plot the results
valid_images = [fp for fp in glob("/content/citypersons/valid/images/*.png")]

for fp_img in np.random.choice(valid_images, 5):
    preds = model.predict(fp_img);
    image = Image.open(fp_img)
    image = np.asarray(image)
    plot_bboxes(image, preds[0].boxes.boxes, score=False)

Output hidden; open in https://colab.research.google.com to view.